In [203]:
!pip install pyvis

In [204]:
import pandas as pd
import networkx as nx
import numpy as np
from pyvis.network import Network
from community import community_louvain

In [205]:
#importing the dataset and showing the head of the table
#filepath_edgelist = 'congress.edgelist'

#data = nx.read_edgelist(filepath_edgelist, create_using=nx.DiGraph)

#G_pandas = nx.to_pandas_edgelist(data)

#G_pandas['source'] = G_pandas['source'].astype(int)
#G_pandas['target'] = G_pandas['target'].astype(int)
#G_pandas.head()

In [206]:
# Importing the dataset and showing the head of the table
df = pd.read_json('https://raw.githubusercontent.com/gsprint23/CongressionalTwitterNetwork/master/congress_network_data.json')
df.head()

,inList,inWeight,outList,outWeight,usernameList
0,"[[4, 9, 11, 13, 18, 27, 29, 33, 38, 45, 46, 50...","[[0.0036496350364960003, 0.0036101083032490005...","[[4, 12, 18, 25, 30, 46, 55, 58, 59, 74, 76, 7...","[[0.002105263157894, 0.002105263157894, 0.0021...","[SenatorBaldwin, SenJohnBarrasso, SenatorBenne..."


In [207]:
# Extracting the necessary data from the dataframe
inList = df['inList'][0]
inWeight = df['inWeight'][0]
outList = df['outList'][0]
outWeight = df['outWeight'][0]
usernameList = df['usernameList'][0]

# Creating empty lists to hold the source, target, and weight values
sources = []
targets = []
weights = []

# Populating the lists with values from inList and outList
for i, (in_edges, out_edges) in enumerate(zip(inList, outList)):
    source = usernameList[i]

    # Handling inbound connections
    for target_index, weight in zip(in_edges, inWeight[i]):
        target = usernameList[target_index]
        sources.append(target)
        targets.append(source)
        weights.append(weight)

    # Handling outbound connections
    for target_index, weight in zip(out_edges, outWeight[i]):
        target = usernameList[target_index]
        sources.append(source)
        targets.append(target)
        weights.append(weight)

# Creating a DataFrame from the populated lists
network_df = pd.DataFrame({
    'source': sources,
    'target': targets,
    'weight': weights
})

# Displaying the first few rows of the new DataFrame to verify the structure
network_df.head()

,source,target,weight
0,SenBlumenthal,SenatorBaldwin,0.003650
1,SenSherrodBrown,SenatorBaldwin,0.003610
2,SenCapito,SenatorBaldwin,0.004073
3,SenatorCarper,SenatorBaldwin,0.003145
4,SenCortezMasto,SenatorBaldwin,0.002347


In [208]:
# Creating a new dataframe called "main_network" with filtered nodes. Here we are choosing the 100 most "weighted" nodes
main_network = network_df.groupby('source').weight.sum().sort_values(ascending=False).index[:100]
main_network

Index(['SpeakerPelosi', 'SteveScalise', 'RepBobbyRush', 'GOPLeader',
       'RepJoeWilson', 'RepMarkTakano', 'RepJamesComer', 'RepBobGood',
       'RepCloudTX', 'RepFranklin', 'RepLucyMcBath', 'SenMikeLee', 'RepSpartz',
       'RepCori', 'PatrickMcHenry', 'rosadelauro', 'RepJohnKatko',
       'RepJeffries', 'replouiegohmert', 'RepKClark', 'RepJudyChu',
       'VernBuchanan', 'RepBarbaraLee', 'RepRalphNorman', 'RepMariaSalazar',
       'RepRichHudson', 'RepTiffany', 'RepBoebert', 'RepWesterman',
       'RepAdamSchiff', 'SenGillibrand', 'SenSchumer', 'SenSanders',
       'BennieGThompson', 'SenTinaSmith', 'RepUnderwood', 'RepMaryMiller',
       'RepDianaDeGette', 'RepBost', 'RepMcKinley', 'RepSpeier', 'RepBlaine',
       'RepAnnWagner', 'cathymcmorris', 'RepJasonCrow', 'SenWarren',
       'SenSherrodBrown', 'RepDavidRouzer', 'RonWyden', 'ChrisVanHollen',
       'RepKatiePorter', 'SenatorLujan', 'RepBice', 'RepHarshbarger',
       'RepArrington', 'RepMcCaul', 'RepRitchie', 'RepLoudermilk'

In [209]:
# Creating edges that will only contain the edges that are related to the nodes in our main_network
edges = network_df[(network_df.source.isin(main_network)) & (network_df.target.isin(main_network))]

In [210]:
# Here we are putting our edgelist into a nx graph
g = nx.from_pandas_edgelist(edges, source='source', target='target', edge_attr='weight')

In [211]:
# Getting an overview of the edges in the nx graph with their "weight" attribute
g.edges(data=True)

EdgeDataView([('SenDuckworth', 'SenSherrodBrown', {'weight': 0.0022675736961450003}), ('SenDuckworth', 'SenAmyKlobuchar', {'weight': 0.006825938566552001}), ('SenDuckworth', 'SenatorLujan', {'weight': 0.0022522522522520003}), ('SenDuckworth', 'SenSchumer', {'weight': 0.006182380216383}), ('SenDuckworth', 'ChrisVanHollen', {'weight': 0.003086419753086}), ('SenDuckworth', 'SenWarren', {'weight': 0.0019267822736030002}), ('SenDuckworth', 'RepBobbyRush', {'weight': 0.006872852233676001}), ('SenDuckworth', 'RepSchneider', {'weight': 0.004926108374384001}), ('SenDuckworth', 'RepJudyChu', {'weight': 0.0022675736961450003}), ('SenDuckworth', 'RepJasonCrow', {'weight': 0.011337868480725}), ('SenDuckworth', 'RepRaulGrijalva', {'weight': 0.0022675736961450003}), ('SenDuckworth', 'RepUnderwood', {'weight': 0.004535147392290001}), ('SenSherrodBrown', 'timkaine', {'weight': 0.005050505050505001}), ('SenSherrodBrown', 'SenAmyKlobuchar', {'weight': 0.0034129692832760004}), ('SenSherrodBrown', 'SenAlex

In [212]:
# Check this shit out do i need it?
#filepath_username = 'congress_network_data.json'

#username_data = pd.read_json(filepath_username)

#def extract_username_list(record):
  #return record

#username_data['Extracted_Username_List'] = username_data['usernameList'].apply(extract_username_list)

#username_list = username_data['usernameList'][0]

#usernames = pd.DataFrame({'username':username_list})
#usernames.head()

In [213]:
#G = nx.from_pandas_edgelist(G_pandas,
                            #source='source',
                            #target='target',
                            #edge_attr='weight',
                            #create_using = nx.DiGraph)
#attributes = usernames.to_dict()['username']
#nx.set_node_attributes(G, attributes, 'username')

In [214]:
centrality_dgr = nx.degree_centrality(g)
dgr = nx.degree(g)
centrality_eig = nx.eigenvector_centrality_numpy(g, weight = 'weight')
centrality_between = nx.betweenness_centrality(g)

In [215]:
dict_dgr = dict(nx.degree(g))
dict_centrality_dgr = dict(nx.degree_centrality(g))
dict_centrality_eig = dict(nx.eigenvector_centrality(g))
dict_centrality_between = dict(nx.betweenness_centrality(g))

In [216]:
nx.set_node_attributes(g, dict_dgr, 'dgr')
nx.set_node_attributes(g, dict_centrality_dgr, 'centrality_dgr')
nx.set_node_attributes(g, dict_centrality_eig, 'centrality_eig')
nx.set_node_attributes(g, dict_centrality_between, 'centrality_between')


In [217]:
#top = 1
#nodes_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
#top_central_nodes = nodes_df[nodes_df.centrality_eig > nodes_df.centrality_eig.quantile(1-top)].index
#G_sub = nx.subgraph(G, top_central_nodes)

In [218]:
G_und = nx.to_undirected(g)
partition = community_louvain.best_partition(g)
nx.set_node_attributes(g, partition, 'community')
nodes_df_und = pd.DataFrame.from_dict(dict(g.nodes(data=True)), orient='index')

In [219]:
net = Network(
    notebook=True,
    cdn_resources="remote",
    bgcolor="#222222",
    font_color="white",
    select_menu=True,
    height="750px",
    width="100%",
    filter_menu=True,
)

net.set_options("""
const options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -23950,
      "centralGravity": 2
    },
    "maxVelocity": 70,
    "minVelocity": 0.75
  }
}
"""
)
net.from_nx(g)

In [220]:
net.show('nx.html')

nx.html
